In [43]:
""" 
    Anirudh Sathish , CS20B1125 
    Question No 4 
"""

' \n    Anirudh Sathish , CS20B1125 \n    Question No 4 \n'

### Question : 
Implement Bayes Classifier for Iris Dataset.
#### Dataset Specifications:
- Total number of samples = 150
- Number of classes = 3 (Iris setosa, Iris virginica, and Iris versicolor)
- Number of samples in each class = 50
#### Use the following information to design classifier:
- Number of training feature vectors ( first 40 in each class) = 40
- Number of test feature vectors ( remaining 10 in each class) = 10
- Number of dimensions = 4
- Feature vector = <sepal length, sepal width, petal length, petal width>

### Approach 
1. Find apriori probability 
2. Find class conditional probability 

In [44]:
# libraries 
import numpy as np 
import pandas as pd

In [45]:
# load data 
iris = pd.read_csv("Iris.csv")
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


- Number of training feature vectors ( first 40 in each class) = 40
- Number of test feature vectors ( remaining 10 in each class)

In [46]:
# split data into test

testData = pd.concat([iris.iloc[:10],iris.iloc[50:60],iris.iloc[100:110]],axis = 0)
trainData = pd.concat([iris.iloc[10:50],iris.iloc[60:100],iris.iloc[110:150]],axis = 0)


In [47]:

flower_classes = set(trainData["Species"])
iris_classes = list(flower_classes)
print(iris_classes)

# Applying the same indexing to train ,test
trainData['Species'] = trainData['Species'].apply(iris_classes.index)
testData['Species'] = testData['Species'].apply(iris_classes.index)

['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']


In [48]:
print(trainData)

      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  Species
10    11            5.4           3.7            1.5           0.2        1
11    12            4.8           3.4            1.6           0.2        1
12    13            4.8           3.0            1.4           0.1        1
13    14            4.3           3.0            1.1           0.1        1
14    15            5.8           4.0            1.2           0.2        1
..   ...            ...           ...            ...           ...      ...
145  146            6.7           3.0            5.2           2.3        0
146  147            6.3           2.5            5.0           1.9        0
147  148            6.5           3.0            5.2           2.0        0
148  149            6.2           3.4            5.4           2.3        0
149  150            5.9           3.0            5.1           1.8        0

[120 rows x 6 columns]


In [49]:
# lets calculate the covariance matrix and mean for the train data

trainingMean_class1 = trainData.iloc[:40,1:5].mean().to_numpy()
trainingCov_class1 = np.cov(trainData.iloc[:40,1:5].to_numpy().T)

trainingMean_class2 = trainData.iloc[40:80,1:5].mean().to_numpy()
trainingCov_class2 = np.cov(trainData.iloc[40:80,1:5].to_numpy().T)

trainingMean_class3 = trainData.iloc[80:120,1:5].mean().to_numpy()
trainingCov_class3 = np.cov(trainData.iloc[80:120,1:5].to_numpy().T)

trainingMean = [trainingMean_class1,trainingMean_class2,trainingMean_class3]
trainingCov = [trainingCov_class1,trainingCov_class2,trainingCov_class3]

# also find inverse of Covs 
trainingInvCov = [np.linalg.inv(x) for x in trainingCov]

In [59]:
def posteriorProbability(dim,covM,X,mean,invCovM):
    X = X.T
    denominator = ((2*np.pi)**(dim/2))*((np.linalg.det(covM))**(1/2))
    power = (-1*(((X-mean).T @ invCovM) @ (X-mean))*(1/2))
    #print(power)
    numerator = np.exp(power)
    posteriorProb = numerator/denominator
    
    return posteriorProb

In [84]:
def classify(dim,covMs,X,means,invCovMs):

    # check across all classes 

    # maintain max var
    max = -1
    flag = -1  
    Maxs = []
    for i in range(0,3):
        Mean = np.resize(means[i],(means[i].shape[0],1))
        posteriorProb = posteriorProbability(dim,covMs[i],X,Mean,invCovMs[i])
        Maxs.append(posteriorProb)
        if posteriorProb > max: 
            max = posteriorProb
            flag = i
    
    # flag is the class
    if flag == 0 :
        return 1
    if flag == 1 :
        return 2  
    if flag == 2 :
        return 0
    return flag 

    

    # check for class 1
   

In [98]:
# running through test data to check for predictions 
dim = 4
predictions = []
for i in range(testData.shape[0]):
    X =  testData.iloc[i:i+1,1:5].to_numpy()
    resultant = classify(dim,trainingCov,X,trainingMean,trainingInvCov)
    predictions.append(resultant)

In [99]:
# checking for accuracy
match = 0  
for i in range(len(testData)):
    expected = testData["Species"].iloc[i]
    prediction = predictions[i]
    if(prediction == expected):
        match+=1

accuracy = (match/len(testData))*100

print("Accuracy of Bayesian Classifier :" ,accuracy)

Accuracy of Bayesian Classifier : 100.0
